In [10]:
import sys
import helpers, folder
from attribute_templates import *
from object_templates import *
from utilities import *
import os
import gemd
from gemd.entity.object import MaterialRun, ProcessRun, IngredientRun, MeasurementRun
from gemd.entity.value import NominalReal
from gemd.entity import PerformedSource, FileLink
from gemd.entity.attribute import Property, Parameter, Condition, PropertyAndConditions
from gemd.json import GEMDJson
from pathlib import Path

In [11]:
encoder = GEMDJson()

### SPECs

In [12]:
from gemd.entity.object import MaterialRun, ProcessRun, IngredientRun, MeasurementRun

In [13]:
# composition_material_spec = generate_composition(conditions=[], 
#                                                     parameters=[], 
#                                                     output_mat_name='Composition', 
#                                                     output_mat_properties=[])
# alloy_material_spec = fabricate_alloy(
#                 ingredients=[composition_material_spec],
#                 conditions=[], 
#                 parameters=[], 
#                 output_mat_name='Alloy', 
#                 output_mat_properties=[])
# alloy_traveler_material_spec = setting_alloy_traveler(
#     ingredients=[alloy_material_spec],
#                 conditions=[], 
#                 parameters=[], 
#                 output_mat_name='Alloy traveler', 
#                 output_mat_properties=[]
# )
# alloy_sample_material_spec = setting_alloy_sample(
#     ingredients=[alloy_traveler_material_spec],
#                 conditions=[], 
#                 parameters=[], 
#                 output_mat_name='Alloy sample', 
#                 output_mat_properties=[]
# )



In [14]:
danial_khatamsaz_performed_source = PerformedSource('danialkh26@tamu.edu', '04/03')

### Modify iteration id below

In [15]:
iteration = "AAA"
path = "/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/"

In [16]:
producible_path = Path(path)
to_display = os.path.join(producible_path, iteration)
paths = folder.Folder.make_tree(Path(to_display))
aggregate = True

In [20]:
%matplotlib inline
import json
import shutil

#TODO: complete details that can't be found without going in for loop (but should be found elsewhere, composition spaceid, composition id, etc)
############ 1] Generating compositions with bayesian optimization
aggregate_summary_sheet_process_spec = ProcessSpec(
            'Aggregate summary sheet',
            template=OBJ_TEMPL['Aggregating'],
            conditions=[],
            parameters=[],
            tags=[],
            file_links=[],
            notes=None
        )
summary_sheet_material_spec = MaterialSpec(
            'Summary Sheet',
            template=OBJ_TEMPL['Summary Sheet'],
            process=aggregate_summary_sheet_process_spec,
            properties=[],
            tags=[],
            file_links=[],
            notes=None
        )
infer_compositions_process_spec = ProcessSpec(
            'Generate the next set of compositions using Bayesian Optimizations',
            template=OBJ_TEMPL['Bayesian Optimization'],
            conditions=[],
            parameters=[],
            tags=[],
            file_links=[],
            notes=None
        )
summary_sheet_ingredient_spec = IngredientSpec(
                '{} Ingredient'.format(summary_sheet_material_spec.name),
                material=summary_sheet_material_spec,
                process=infer_compositions_process_spec,
                tags = [],
                file_links=[],
                notes=None,
                labels=[],
            )
# is the compositional space fixed?
suggested_compositions_material_spec = MaterialSpec(
            'Suggested Compositions',
            template=OBJ_TEMPL['Inferred Alloy Compositions'],
            process=infer_compositions_process_spec,
            properties=[],
            tags=[],
            file_links=[],
            notes=None
        )


count = 0
for path in paths:
    print(path.path)
    if '/DED' in str(path.path) and path.depth == 5: # example: /data/AAA/DED/A/AAA01-AAA08/AAA01/C01
        continue
    if '/VAM' in str(path.path) and path.depth == 4: # example: /data/AAA/VAM/B/AAA01/T01
        
        vam_processing_details = json.load(open(os.path.join(path.path, 'vam-processing-details-v1.json')))
        vam_synthesis_details = json.load(open(os.path.join(path.path, 'vam-synthesis-details-v1.json')))
        vam_traveler = json.load(open(os.path.join(path.path, 'vam-traveler-v1.json')))
        full_path = str(path.path)
        composition_space_id = full_path.split('/')[1]
        composition_id = full_path.split('/')[-2]
        if count == 1:
            print(composition_id)
            # print(json.dumps(vam_processing_details, indent=4))
            # print(json.dumps(vam_synthesis_details, indent=4))
            # print(json.dumps(vam_traveler, indent=4))
        raw_jsons_dirpath = os.path.join(composition_id,'raw_jsons' )
        thin_jsons_dirpath = os.path.join(composition_id,'thin_jsons' )
        if os.path.exists(composition_id):
            shutil.rmtree(composition_id)
        os.makedirs(composition_id)
        os.makedirs(raw_jsons_dirpath)
        os.makedirs(thin_jsons_dirpath)
        mat_composition = vam_traveler['data']['Material Composition']
        batch_id = vam_traveler['data']['Sample ID']['Prod. Batch']
        yymm = vam_traveler['data']['Sample ID']['Year & Month']
        # print(mat_composition['Al'])
        # print(type(mat_composition['Al']))
        
        ############ 2] Selecting composition
        select_composition_process_spec = ProcessSpec(
            'Selecting composition {} for batch {} '.format(composition_id, batch_id),
            template=OBJ_TEMPL['Selecting'],
            conditions=[],
            parameters=[],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        suggested_compositions_ingredient_spec = IngredientSpec(
                '{} Ingredient'.format(suggested_compositions_material_spec.name),
                material=suggested_compositions_material_spec,
                process=select_composition_process_spec,
                tags = [],
                file_links=[],
                notes=None,
                labels=[],
            )
        composition_material_spec = MaterialSpec(
            'Composition {} in batch {} '.format(composition_id, batch_id),
            template=OBJ_TEMPL['Alloy Composition'],
            process=select_composition_process_spec,
            properties=[
                    PropertyAndConditions(property=Property('Al', value=NominalReal(float(mat_composition['Al']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
                    PropertyAndConditions(property=Property('Co', value=NominalReal(float(mat_composition['Co']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
                    PropertyAndConditions(property=Property('Cr', value=NominalReal(float(mat_composition['Cr']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
                    PropertyAndConditions(property=Property('Fe', value=NominalReal(float(mat_composition['Fe']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
                    PropertyAndConditions(property=Property('Ni', value=NominalReal(float(mat_composition['Ni']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
                    PropertyAndConditions(property=Property('V', value=NominalReal(float(mat_composition['V']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
            ],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        
        ############ 3] Weighting + aggregating or buying materials 
        elements = []
        if aggregate:
            for p in composition_material_spec.properties:
                aggregating_material_process_spec = ProcessSpec(
                    'Aggregating {} for batch {} '.format(p.name, batch_id),
                    template=OBJ_TEMPL['Aggregating'],
                    conditions=[],
                    parameters=[],
                    tags=[composition_id, yymm, batch_id],
                    file_links=[],
                    notes=None
                )
                composition_element_material_spec = MaterialSpec(
                    '{}'.format(p.name),
                    template=OBJ_TEMPL['Composition Element'],
                    process=aggregating_material_process_spec,
                    properties=[],
                    tags=[composition_id, yymm, batch_id],
                    file_links=[],
                    notes=None
                )
                elements.append(composition_element_material_spec)
                
                IngredientSpec(
                    '{} Ingredient'.format(composition_material_spec.name),
                    material=composition_material_spec,
                    process=aggregating_material_process_spec,
                    tags=[composition_id, yymm, batch_id],
                    file_links=[],
                    notes=None,
                    labels=[],
                )
                
                # measurement: weighted mass
                weighting_measurement_spec = MeasurementSpec(
                    name='Weighting {}'.format(p.name),
                    template=OBJ_TEMPL['Weighting'],
                    notes=[],
                    tags=[composition_id, yymm, batch_id],
                    file_links=[],
                    parameters=[],
                    conditions=[]
                )
        
        ############ 4] mix
        mixing_process_spec = ProcessSpec(
            'Mixing individual elements of batch {} for composition {}'.format(batch_id, composition_id),
            template=OBJ_TEMPL['Mixing'],
            conditions=[],
            parameters=[],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        alloy_material_spec = MaterialSpec(
                    '{} Alloy (Batch {})'.format(composition_id, batch_id),
                    template=OBJ_TEMPL['Alloy'],
                    process=mixing_process_spec,
                    properties=[],
                    tags=[composition_id, yymm, batch_id],
                    file_links=[],
                    notes=None,
                    
                )
        for e in elements:
            IngredientSpec(
                f'{e.name} Ingredient',
                material=e,
                process=mixing_process_spec,
                tags=[composition_id, yymm, batch_id],
                file_links=[],
                notes=None,
                labels=[])
        
        arc_melting_process_spec = ProcessSpec(
            'Arc Melting for batch {} of {} '.format(batch_id, composition_id),
            template=OBJ_TEMPL['Arc Melting'],
            conditions=[],
            parameters=[],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        melted_alloy_material_spec = MaterialSpec(
            '{} Alloy (melted)'.format(composition_id),
            template=OBJ_TEMPL['Alloy'],
            process=arc_melting_process_spec,
            properties=[],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        IngredientSpec(
            '{} Ingredient'.format(alloy_material_spec.name),
            material=alloy_material_spec,
            process=arc_melting_process_spec,
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None,
            labels=[],
        )
        
        ############ 7) Homogenization
        homonegization_process_spec = ProcessSpec(
            'homogenizing batch {} of {}'.format(batch_id, composition_id),
            template=OBJ_TEMPL['Homogenization'],
            conditions=[],
            parameters=[],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        homonegized_alloy_material_spec = MaterialSpec(
            '{} Alloy (homogenized)'.format(composition_id),
            template=OBJ_TEMPL['Alloy'],
            process=homonegization_process_spec,
            properties=[],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        IngredientSpec(
            '{} Ingredient'.format(melted_alloy_material_spec.name),
            material=melted_alloy_material_spec,
            process=homonegization_process_spec,
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None,
            labels=[],
        )
        
        ############ 8) Forging
        forging_process_spec = ProcessSpec(
                    'Forging batch {} of {}'.format(batch_id, composition_id),
                    template=OBJ_TEMPL['Forging'],
                    conditions=[],
                    parameters=[],
                    tags=[composition_id, yymm, batch_id],
                    file_links=[],
                    notes=None
                )
        forged_alloy_material_spec = MaterialSpec(
            '{} Alloy (forged)'.format(composition_id),
            template=OBJ_TEMPL['Alloy'],
            process=forging_process_spec,
            properties=[],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        IngredientSpec(
            '{} Ingredient'.format(homonegized_alloy_material_spec.name),
            material=homonegized_alloy_material_spec,
            process=forging_process_spec,
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None,
            labels=[],
        )
        
        ############ 6] creating the traveler for the composition
        
        ############ 7] create traveler samples
        
        ############ 8] characterization/measurements
        
        # print(composition_id)
        helpers.test(encoder, composition_id, forged_alloy_material_spec, mode="spec")
        # if count == 0:
        # count += 1 
        break


/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/AAA/VAM/B/AAA02/T01/T03
/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/AAA/VAM/B/AAA02/T01/T03/Microhardness-TAMU
/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/AAA/VAM/B/AAA02/T01/T07
/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/AAA/VAM/B/AAA02/T01/T07/T11
/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/AAA/VAM/B/AAA02/T01/T07/T11/Tensile
/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/AAA/VAM/B/AAA02/T01/vam-processing-details-v1.json
/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/AAA/VAM/B/AAA02/T01/vam-synthesis-details-v1.json
/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/AAA/VAM/B/AAA02/T01/vam-traveler-v1.json
/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/AAA/VAM/B/AAA03
/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/AAA/VAM/B/AAA03/T01
cycles in the graph: []
nb of disregarded elements (i.e., templates/specs): 1/56


In [18]:
# generate_composition_process_run = ProcessRun(
#             name = "Generating composition: {}".format(composition_id),
#             spec = composition_material_spec.process,
#             source = danial_khatamsaz_performed_source,
#             tags = []   
#         )
#         composition_material_run = MaterialRun(
#             name = "Composition: {}".format(composition_id),
#             process = generate_composition_process_run,
#             spec = composition_material_spec,
#             sample_type = 'experimental',
#             tags = []
#         )
#         # fab